In [5]:
import torch

In [6]:
import argparse
import os
import random
from glob import glob

random.seed(0)

def main(args):
    for split in ['train', 'test']:
        samples = []
        for class_label in ['pos', 'neg']:
            fnames = glob(os.path.join(args.datadir, split, class_label) + '/*.txt')
            for fname in fnames:
                with open(fname) as fin:
                    line = fin.readline()
                    samples.append((line, 1 if class_label == 'pos' else 0))
        random.shuffle(samples)
        out_fname = 'train' if split == 'train' else 'dev'
        f1 = open(os.path.join(args.datadir, out_fname + '.input0'), 'w')
        f2 = open(os.path.join(args.datadir, out_fname + '.label'), 'w')
        for sample in samples:
            f1.write(sample[0] + '\n')
            f2.write(str(sample[1]) + '\n')
        f1.close()
        f2.close()

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--datadir', default='aclImdb')
    args = parser.parse_args()
    main(args)

usage: ipykernel_launcher.py [-h] [--datadir DATADIR]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/dani/Library/Jupyter/runtime/kernel-a4cf0582-43a1-45ea-811a-e5ee364dbf87.json


SystemExit: 2

/Users/dani/Desktop/MLOPS_Team4/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [36]:
import argparse
import os
import random
from glob import glob
import csv

# Interim -> processed (dev.input0, dev.label, train.input0, train.label)

In [35]:
import csv
for split in ['train', 'test']:
    samples = []
    args_datadir = "~/Desktop/MLOPS_Team4/data" ## ARGS later
    csv_path = os.path.join(os.path.expanduser(args_datadir), "interim", f"{split}_tweets.csv")
    with open(csv_path, mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for line in csv_reader:
            if line[0] == 'pos':
                label = 1
            elif line[0] == 'neg':
                label = 0
            elif line[0] == 'neu':
                label = 2
            else:
                continue  # Skip lines with unexpected labels
            samples.append((line[1], label))
    random.shuffle(samples)
    out_fname = 'train' if split == 'train' else 'dev'

    f1 = open(os.path.join(os.path.expanduser(args_datadir), "processed", out_fname + '.input0'), 'w')
    f2 = open(os.path.join(os.path.expanduser(args_datadir), "processed", out_fname + '.label'), 'w')
    for sample in samples:
        f1.write(sample[0] + '\n')
        f2.write(str(sample[1]) + '\n')
    f1.close()
    f2.close()

# Multiprocessing BPE encoder

In [6]:
encoder_json = "/Users/dani/Desktop/MLOPS_Team4/data/external/encoder.json"
vocab_bpe = "/Users/dani/Desktop/MLOPS_Team4/data/external/vocab.bpe" 
inputs_args = ["/Users/dani/Desktop/MLOPS_Team4/data/processed/train.input0"] 
outputs_args = ["/Users/dani/Desktop/MLOPS_Team4/data/processed/train.input0.bpe"]
workers = 1 
keep_empty = True

import argparse
import contextlib
import sys
from collections import Counter
from multiprocessing import Pool
from fairseq.data.encoders.gpt2_bpe import get_encoder


def encode(line):
        global bpe
        ids = bpe.encode(line)
        return list(map(str, ids))
    
def encode_lines(lines):
        """
        Encode a set of lines. All lines will be encoded together.
        """
        enc_lines = []
        for line in lines:
            line = line.strip()
            if len(line) == 0 and not keep_empty:
                return ["EMPTY", None]
            tokens = encode(line)
            enc_lines.append(" ".join(tokens))
        return ["PASS", enc_lines]

with contextlib.ExitStack() as stack:
        inputs = [
            stack.enter_context(open(input, "r", encoding="utf-8"))
            if input != "-"
            else sys.stdin
            for input in inputs_args
        ]
        outputs = [
            stack.enter_context(open(output, "w", encoding="utf-8"))
            if output != "-"
            else sys.stdout
            for output in outputs_args
        ]

#encoder = MultiprocessingEncoder(args)
global bpe
bpe = get_encoder(encoder_json, vocab_bpe)
pool = Pool(workers, initializer=bpe)
encoded_lines = pool.imap(encode_lines, zip(*inputs), 100)

stats = Counter()
for i, (filt, enc_lines) in enumerate(encoded_lines, start=1):
    if filt == "PASS":
        for enc_line, output_h in zip(enc_lines, outputs):
            print(enc_line, file=output_h)
    else:
        stats["num_filtered_" + filt] += 1
    if i % 10000 == 0:
        print("processed {} lines".format(i), file=sys.stderr)

for k, v in stats.most_common():
    print("[{}] filtered {} lines".format(k, v), file=sys.stderr)

ModuleNotFoundError: No module named 'fairseq'

In [7]:
!pip install fairseq==0.12.1

  Using cached fairseq-0.12.1.tar.gz (9.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      Traceback (most recent call last):
        File "/Users/dani/Desktop/MLOPS_Team4/venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/Users/dani/Desktop/MLOPS_Team4/venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/dani/Desktop/MLOPS_Team4/venv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settin